In [1]:
%cd ../..

/home/work/AnacondaProjects/combinators


In [2]:
import logging

import numpy as np
import probtorch
import torch

from examples.fep_control import fep_control
from combinators.model import active
from combinators.model import compose, foldable
from combinators.inference import importance
from combinators import utils

In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [4]:
target_actor = fep_control.MountainCarActor(state_dim=2, batch_shape=(1,), trainable=False)
target_observer = fep_control.GenerativeObserver(observation_dim=2, batch_shape=(1,), trainable=False)
target = compose(target_observer, target_actor)

proposal_actor = fep_control.RecognitionActor(state_dim=2, action_dim=1, observation_dim=2, batch_shape=(1,),
                                              discrete_actions=False, name='MountainCarActor', trainable=True)
proposal_encoder = fep_control.RecognitionEncoder(state_dim=2, observation_dim=2, batch_shape=(1,), trainable=True)
proposal = compose(proposal_encoder, proposal_actor)

agent = importance.propose(target, proposal)

In [5]:
theta, graph, log_weight = active.active_inference_test(agent, 'MountainCarContinuous-v0', online_inference=False, iterations=1000)

In [6]:
theta, graph, log_weight = active.active_inference(agent, 'MountainCarContinuous-v0', lr=1e-2, episodes=100, episode_length=1000, dream=False)

06/06/2019 16:47:10 ELBO=-1.09549367e+05 at episode 0 of length 1000
06/06/2019 16:47:23 ELBO=-6.49738984e+04 at episode 1 of length 1000
06/06/2019 16:47:35 ELBO=-2.22765273e+04 at episode 2 of length 303
06/06/2019 16:47:48 ELBO=-9.49189453e+03 at episode 3 of length 1000
06/06/2019 16:48:01 ELBO=-8.50049512e+03 at episode 4 of length 370
06/06/2019 16:48:15 ELBO=-3.39635078e+04 at episode 5 of length 979
06/06/2019 16:48:28 ELBO=-2.39988223e+04 at episode 6 of length 1000
06/06/2019 16:48:41 ELBO=-1.21972783e+04 at episode 7 of length 1000
06/06/2019 16:48:54 ELBO=-5.92782129e+03 at episode 8 of length 700
06/06/2019 16:49:07 ELBO=-4.70251904e+03 at episode 9 of length 273
06/06/2019 16:49:20 ELBO=-6.60590381e+03 at episode 10 of length 542
06/06/2019 16:49:33 ELBO=-9.70266895e+03 at episode 11 of length 770
06/06/2019 16:49:44 ELBO=-9.56446680e+03 at episode 12 of length 666
06/06/2019 16:49:54 ELBO=-7.82974268e+03 at episode 13 of length 337
06/06/2019 16:50:05 ELBO=-1.26402148e+0

In [7]:
log_weight

tensor([-4111.9946], device='cuda:0', grad_fn=<AddBackward0>)

In [12]:
theta, graph, log_weight = active.active_inference_test(agent, 'MountainCarContinuous-v0', online_inference=False, iterations=1000)

In [9]:
theta

(tensor([[0.1552, 0.2662]], device='cuda:0', grad_fn=<AddBackward0>),
 tensor([[0.8682]], device='cuda:0', grad_fn=<AddBackward0>))

In [10]:
log_weight

tensor([-3030.3499], device='cuda:0', grad_fn=<AddBackward0>)

In [11]:
torch.save(agent, 'examples/fep_control/fep_mountain_car_agent.dat')